In [1]:
import pandas as pd
import numpy as np

#read data from csv
data = pd.read_csv("data/training_data_utf8.csv")

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/loherzberger/Projects/ML/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
#define function to transform long/lat to distance
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km

In [3]:
#define long/lat center of vienna
latitude_center = 48.210033
longitude_center = 16.363449

In [4]:
#create new column distance_to_center
for index, row in data.iterrows():
    data.loc[index, 'distance_to_center'] = haversine(row['longitude'], row['latitude'], longitude_center, latitude_center)

In [5]:
#extract year from transaction_ym
date = data.transaction_ym.apply(pd.to_datetime, format="%Y-%m")

data["year"] = date.dt.year

In [6]:
#create new column relative_share
relative_share = data.plot_area * data.plot_share
data["relative_share"] = relative_share

In [7]:
#filter outliers
data = data[data.distance_to_center < 15]
data = data[data.price_per_m2 < 2500]
data.shape

(30441, 21)

In [8]:
#fill NA/NaN with mean
data = data.fillna(data.mean())

In [9]:
data["land_use"] = data["land_use"].astype(str)
data["district"] = data["district"].astype(str)
data["property_type"] = data["property_type"].astype(str)
data["contract_type"] = data["contract_type"].astype(str)
data["protection_zone"] = data["protection_zone"].astype(str)
data["building_ban"] = data["building_ban"].astype(str)

In [10]:
#split data into training and test data
data_train = data[data.year <= 2010]
data_test = data[data.year > 2010]

In [11]:
#display shape of training data
data_train.shape

(27754, 21)

In [12]:
#display shape of testing data
data_test.shape

(2687, 21)

In [13]:
#define numerical and categorical pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from future_encoders import OneHotEncoder

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('one_hot', OneHotEncoder())
])

In [14]:
#define features for pipeline A
pip_data_a = data[["building_floors", "year", "distance_to_center", "land_use", "relative_share", "district"]]
pip_data_a_num = data[["year", "distance_to_center", "building_floors", "relative_share"]]
pip_data_a_cat = data[["land_use", "district"]]

In [42]:
#define full_pipeline_a for training data_a
from future_encoders import ColumnTransformer

num_attribs = list(pip_data_a_num)
cat_attribs = list(pip_data_a_cat)

full_pipeline_a = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', cat_pipeline, cat_attribs)
])

full_pipeline_a.fit_transform(pip_data_a)

<30441x75 sparse matrix of type '<class 'numpy.float64'>'
	with 182646 stored elements in Compressed Sparse Row format>

In [16]:
#define training data for pipeline A
data_train_a = data_train[["price_per_m2", "building_floors", "year", "distance_to_center", "land_use", "relative_share", "district"]]

data_train_labels_a = data_train_a["price_per_m2"].copy()
data_train_a = data_train_a.drop("price_per_m2", axis=1)

In [17]:
#define features for pipeline B
pip_data_b = data[["price_per_m2", "property_type","district","contract_type","protection_zone","building_ban", "building_floors", "building_year", "land_use"]]
pip_data_b_num = data[["building_floors", "building_year"]]
pip_data_b_cat = data[["property_type","district","contract_type","protection_zone","building_ban", "land_use"]]

In [18]:
#define full_pipeline_b for training data_b
num_attribs = list(pip_data_b_num)
cat_attribs = list(pip_data_b_cat)

full_pipeline_b = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', cat_pipeline, cat_attribs)
])

full_pipeline_b.fit_transform(pip_data_b)

<30441x104 sparse matrix of type '<class 'numpy.float64'>'
	with 243528 stored elements in Compressed Sparse Row format>

In [19]:
#define training data for pipeline B
data_train_b = data_train[["price_per_m2", "property_type","district","contract_type","protection_zone","building_ban", "building_floors", "building_year", "land_use"]]
data_train_b_num = data_train[["building_floors", "building_year"]]
data_train_b_cat = data_train[["property_type","district","contract_type","protection_zone","building_ban", "land_use"]]

data_train_labels_b = data_train_b["price_per_m2"].copy()
data_train_b = data_train_b.drop("price_per_m2", axis=1)

In [20]:
#fit data_a and data_b
data_train_prep_a = full_pipeline_a.transform(data_train_a)
data_train_prep_b = full_pipeline_b.transform(data_train_b)

In [21]:
#declare diplay_scores function for model evaluation
from sklearn.model_selection import cross_val_score

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard Deviation", scores.std())

In [22]:
#train and cv linear regression for data_prep_a
from sklearn.linear_model import LinearRegression

lin_reg_a = LinearRegression()
lin_reg_a.fit(data_train_prep_a, data_train_labels_a)

scores = cross_val_score(lin_reg_a, data_train_prep_a, data_train_labels_a, scoring="neg_mean_squared_error", cv=10)
lin_reg_rmse_scores_a = np.sqrt(-scores)

display_scores(lin_reg_rmse_scores_a)

Scores: [428.07599805 383.70809955 368.58897131 323.70836266 342.89403222
 344.57305531 359.15661085 371.24994065 308.75595641 336.02724009]
Mean: 356.67382670940106
Standard Deviation 32.14833902938146


In [23]:
#train and cv linear regression for data_prep_b
lin_reg_b = LinearRegression()
lin_reg_b.fit(data_train_prep_b, data_train_labels_b)

scores = cross_val_score(lin_reg_b, data_train_prep_b, data_train_labels_b, scoring="neg_mean_squared_error", cv=10)
lin_reg_rmse_scores_b = np.sqrt(-scores)

display_scores(lin_reg_rmse_scores_b)

Scores: [453.99782919 390.94191389 380.43655399 339.26626396 354.64263696
 348.94466852 352.93208611 352.41580021 348.31558628 422.60774121]
Mean: 374.4501080345841
Standard Deviation 35.87078229619428


In [24]:
#train and cv ridge regression for data_prep_a
from sklearn.linear_model import Ridge

ridge_reg_a = Ridge(alpha=0.9)
ridge_reg_a.fit(data_train_prep_a, data_train_labels_a)

scores = cross_val_score(ridge_reg_a, data_train_prep_a, data_train_labels_a, scoring="neg_mean_squared_error", cv=10)
ridge_reg_rmse_scores_a = np.sqrt(-scores)

display_scores(ridge_reg_rmse_scores_a)

Scores: [428.36068786 383.92625523 367.82797659 323.52101959 342.88948522
 344.667256   359.09334766 371.17649736 308.9600659  336.16981695]
Mean: 356.6592408360203
Standard Deviation 32.175436608183915


In [25]:
#train and cv ridge regression for data_prep_b
ridge_reg_b = Ridge(alpha=0.9)
ridge_reg_b.fit(data_train_prep_b, data_train_labels_b)

scores = cross_val_score(ridge_reg_b, data_train_prep_b, data_train_labels_b, scoring="neg_mean_squared_error", cv=10)
ridge_reg_rmse_scores_b = np.sqrt(-scores)

display_scores(ridge_reg_rmse_scores_b)

Scores: [453.93092758 390.82498309 380.30410339 339.27748146 354.62062984
 348.96406242 352.90269641 352.45015978 348.27731046 422.39064186]
Mean: 374.39429962969984
Standard Deviation 35.820433412764665


In [26]:
#train and cv lasso regression for data_prep_a
from sklearn.linear_model import Lasso

lasso_reg_a = Lasso(alpha=0.5)
lasso_reg_a.fit(data_train_prep_a, data_train_labels_a)

scores = cross_val_score(lasso_reg_a, data_train_prep_a, data_train_labels_a, scoring="neg_mean_squared_error", cv=10)
lasso_reg_rmse_scores_a = np.sqrt(-scores)

display_scores(lasso_reg_rmse_scores_a)

Scores: [432.20985016 387.39643375 366.81390533 323.0161888  344.04912034
 348.6945086  360.52429909 373.53331489 310.38127547 335.51775564]
Mean: 358.2136652071593
Standard Deviation 33.12735581313222


In [27]:
#train and cv lasso regression for data_prep_b
lasso_reg_b = Lasso(alpha=0.5)
lasso_reg_b.fit(data_train_prep_b, data_train_labels_b)

scores = cross_val_score(lasso_reg_b, data_train_prep_b, data_train_labels_b, scoring="neg_mean_squared_error", cv=10)
lasso_reg_rmse_scores_b = np.sqrt(-scores)

display_scores(lasso_reg_rmse_scores_b)

Scores: [454.327219   390.26059253 382.62988832 340.34179351 354.97732535
 351.94100345 353.38416872 354.67759394 349.51501726 419.50789407]
Mean: 375.1562496150731
Standard Deviation 34.96878871799998


In [28]:
#train and cv decision tree regression for data_prep_a
from sklearn.tree import DecisionTreeRegressor

tree_reg_a = DecisionTreeRegressor()
tree_reg_a.fit(data_train_prep_a, data_train_labels_a)

scores = cross_val_score(tree_reg_a, data_train_prep_a, data_train_labels_a, scoring="neg_mean_squared_error", cv=10)
tree_reg_rmse_scores_a = np.sqrt(-scores)

display_scores(tree_reg_rmse_scores_a)

Scores: [539.26169024 487.32458726 466.91962114 452.94526351 438.44682117
 450.24779663 442.72145787 463.27005546 420.1088163  371.01117005]
Mean: 453.22572796292934
Standard Deviation 41.2880080855494


In [29]:
#train and cv decision tree regression for data_prep_b
tree_reg_b = DecisionTreeRegressor()
tree_reg_b.fit(data_train_prep_b, data_train_labels_b)

scores = cross_val_score(tree_reg_b, data_train_prep_b, data_train_labels_b, scoring="neg_mean_squared_error", cv=10)
tree_reg_rmse_scores_b = np.sqrt(-scores)

display_scores(tree_reg_rmse_scores_b)

Scores: [451.97221121 427.65506722 443.12416704 396.22633096 419.01510951
 396.84902032 404.41775304 407.98584504 392.87356165 445.19444834]
Mean: 418.53135143095415
Standard Deviation 21.104159761313884


In [30]:
#train and cv random forest regression for data_prep_a
from sklearn.ensemble import RandomForestRegressor

forest_reg_a = RandomForestRegressor()
forest_reg_a.fit(data_train_prep_a, data_train_labels_a)

scores = cross_val_score(forest_reg_a, data_train_prep_a, data_train_labels_a, scoring="neg_mean_squared_error", cv=10)
forest_reg_rmse_scores_a = np.sqrt(-scores)

display_scores(forest_reg_rmse_scores_a)

/home/loherzberger/Projects/ML/venv/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Scores: [397.89424775 360.54713375 351.90751428 329.60500394 331.21194746
 335.13000581 348.12351399 373.75023834 321.63884779 293.27661395]
Mean: 344.308506705876
Standard Deviation 27.68458896281967


In [31]:
#train and cv random forest regression for data_prep_b
forest_reg_b = RandomForestRegressor()
forest_reg_b.fit(data_train_prep_b, data_train_labels_b)

scores = cross_val_score(forest_reg_b, data_train_prep_b, data_train_labels_b, scoring="neg_mean_squared_error", cv=10)
forest_reg_rmse_scores_b = np.sqrt(-scores)

display_scores(forest_reg_rmse_scores_b)

Scores: [419.24656918 377.61920628 398.53856958 345.34002215 354.1344575
 371.26651435 380.81338225 378.55221163 347.66025028 413.53672732]
Mean: 378.67079105198195
Standard Deviation 24.47838897280553


In [32]:
#prepare test_data_a for testing
test_data_a = data_test[["price_per_m2", "building_floors", "year", "distance_to_center", "land_use", "relative_share", "district"]]

test_labels_a = test_data_a["price_per_m2"].copy()
test_set_a = test_data_a.drop("price_per_m2", axis=1)

test_data_prep_a = full_pipeline_a.transform(test_set_a)

#prepare test_data_b for testing
test_data_b = data_test[["price_per_m2", "property_type","district","contract_type","protection_zone","building_ban", "building_floors", "building_year", "land_use"]]

test_labels_b = test_data_b["price_per_m2"].copy()
test_set_b = test_data_b.drop("price_per_m2", axis=1)

test_data_prep_b = full_pipeline_b.transform(test_set_b)

In [33]:
#prepare predictions on test set

#predictions from A-Models
lin_reg_pred_a = lin_reg_a.predict(test_data_prep_a)
ridge_reg_pred_a = ridge_reg_a.predict(test_data_prep_a)
lasso_reg_pred_a = lasso_reg_a.predict(test_data_prep_a)
tree_reg_pred_a = tree_reg_a.predict(test_data_prep_a)
forest_reg_pred_a = forest_reg_a.predict(test_data_prep_a)

#predictions from B-Models
lin_reg_pred_b = lin_reg_b.predict(test_data_prep_b)
ridge_reg_pred_b = ridge_reg_b.predict(test_data_prep_b)
lasso_reg_pred_b = lasso_reg_b.predict(test_data_prep_b)
tree_reg_pred_b = tree_reg_b.predict(test_data_prep_b)
forest_reg_pred_b = forest_reg_b.predict(test_data_prep_b)

In [34]:
#used metrics for experiment
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [35]:
#get metrics for models

#metrics for A-Models
lin_mse_a = mean_squared_error(test_labels_a, lin_reg_pred_a)
lin_mae_a = mean_absolute_error(test_labels_a, lin_reg_pred_a)
ridge_mse_a = mean_squared_error(test_labels_a, ridge_reg_pred_a)
ridge_mae_a = mean_absolute_error(test_labels_a, ridge_reg_pred_a)
lasso_mse_a = mean_squared_error(test_labels_a, lasso_reg_pred_a)
lasso_mae_a = mean_absolute_error(test_labels_a, lasso_reg_pred_a)
tree_mse_a = mean_squared_error(test_labels_a, tree_reg_pred_a)
tree_mae_a = mean_absolute_error(test_labels_a, tree_reg_pred_a)
forest_mse_a = mean_squared_error(test_labels_a, forest_reg_pred_a)
forest_mae_a = mean_absolute_error(test_labels_a, forest_reg_pred_a)


#metrics for B-Models
lin_mse_b = mean_squared_error(test_labels_b, lin_reg_pred_b)
lin_mae_b = mean_absolute_error(test_labels_b, lin_reg_pred_b)
ridge_mse_b = mean_squared_error(test_labels_b, ridge_reg_pred_b)
ridge_mae_b = mean_absolute_error(test_labels_b, ridge_reg_pred_b)
lasso_mse_b = mean_squared_error(test_labels_b, lasso_reg_pred_b)
lasso_mae_b = mean_absolute_error(test_labels_b, lasso_reg_pred_b)
tree_mse_b = mean_squared_error(test_labels_b, tree_reg_pred_b)
tree_mae_b = mean_absolute_error(test_labels_b, tree_reg_pred_b)
forest_mse_b = mean_squared_error(test_labels_b, forest_reg_pred_b)
forest_mae_b = mean_absolute_error(test_labels_b, forest_reg_pred_b)

In [36]:
df_a = pd.DataFrame(columns=['Linear/MSE', 'Linear/MAE', 'Ridge/MSE', 'Ridge/MAE', 'Lasso/MSE', 'Lasso/MAE', 'Tree/MSE', 'Tree/MAE', 'Forest/MSE', 'Forest/MAE'])
df_a['Linear/MSE'] = [lin_mse_a]
df_a['Linear/MAE'] = [lin_mae_a]
df_a['Ridge/MSE'] = [ridge_mse_a]
df_a['Ridge/MAE'] = [ridge_mae_a]
df_a['Lasso/MSE'] = [lasso_mse_a]
df_a['Lasso/MAE'] = [lasso_mae_a]
df_a['Tree/MSE'] = [tree_mse_a]
df_a['Tree/MAE'] = [tree_mae_a]
df_a['Forest/MSE'] = [forest_mse_a]
df_a['Forest/MAE'] = [forest_mae_a]

df_a

,Linear/MSE,Linear/MAE,Ridge/MSE,Ridge/MAE,Lasso/MSE,Lasso/MAE,Tree/MSE,Tree/MAE,Forest/MSE,Forest/MAE
0,216558.587898,343.656663,216794.697258,344.0786,219044.208201,346.466355,280586.307571,368.740114,183794.900856,297.953178


In [37]:
df_b = pd.DataFrame(columns=['Linear/MSE', 'Linear/MAE', 'Ridge/MSE', 'Ridge/MAE', 'Lasso/MSE', 'Lasso/MAE', 'Tree/MSE', 'Tree/MAE', 'Forest/MSE', 'Forest/MAE'])
df_b['Linear/MSE'] = [lin_mse_b]
df_b['Linear/MAE'] = [lin_mae_b]
df_b['Ridge/MSE'] = [ridge_mse_b]
df_b['Ridge/MAE'] = [ridge_mae_b]
df_b['Lasso/MSE'] = [lasso_mse_b]
df_b['Lasso/MAE'] = [lasso_mae_b]
df_b['Tree/MSE'] = [tree_mse_b]
df_b['Tree/MAE'] = [tree_mae_b]
df_b['Forest/MSE'] = [forest_mse_b]
df_b['Forest/MAE'] = [forest_mae_b]

df_b

,Linear/MSE,Linear/MAE,Ridge/MSE,Ridge/MAE,Lasso/MSE,Lasso/MAE,Tree/MSE,Tree/MAE,Forest/MSE,Forest/MAE
0,248655.957769,339.963574,248543.291939,339.78874,250681.552118,342.080145,249343.643634,334.718229,219979.578944,317.602263


In [40]:
sorted(zip(tree_reg_b.feature_importances_, num_attribs + cat_attribs), reverse=True)

[(0.3021688574808543, 'building_floors'),
 (0.15925461969551222, 'building_year'),
 (0.0074118668304055235, 'contract_type'),
 (0.006052116828774271, 'property_type'),
 (0.004279947609165976, 'building_ban'),
 (0.003622788305831662, 'district'),
 (0.0008557226727345851, 'protection_zone'),
 (0.0, 'land_use')]

In [43]:
sorted(zip(tree_reg_a.feature_importances_, num_attribs + cat_attribs), reverse=True)

[(0.4288235487396782, 'distance_to_center'),
 (0.1821114197260744, 'year'),
 (0.160040708816696, 'relative_share'),
 (0.0871539720064223, 'building_floors'),
 (0.00010981242060749826, 'land_use'),
 (0.0, 'district')]

In [41]:
sorted(zip(forest_reg_b.feature_importances_, num_attribs + cat_attribs), reverse=True)

[(0.297501665522286, 'building_floors'),
 (0.17039672245813714, 'building_year'),
 (0.006196497132072714, 'property_type'),
 (0.0059084139932619835, 'contract_type'),
 (0.00511233604846519, 'building_ban'),
 (0.004039597495434979, 'district'),
 (0.0009981690207341094, 'protection_zone'),
 (5.359427084232771e-06, 'land_use')]

In [44]:
sorted(zip(forest_reg_a.feature_importances_, num_attribs + cat_attribs), reverse=True)

[(0.42822532283669945, 'distance_to_center'),
 (0.1814231643861969, 'year'),
 (0.15358685652667442, 'relative_share'),
 (0.09177652367580746, 'building_floors'),
 (0.00010555361184959944, 'land_use'),
 (7.329625804671939e-08, 'district')]

In [46]:
pd.concat([df_a, df_b])

,Linear/MSE,Linear/MAE,Ridge/MSE,Ridge/MAE,Lasso/MSE,Lasso/MAE,Tree/MSE,Tree/MAE,Forest/MSE,Forest/MAE
0,216558.587898,343.656663,216794.697258,344.07860,219044.208201,346.466355,280586.307571,368.740114,183794.900856,297.953178
0,248655.957769,339.963574,248543.291939,339.78874,250681.552118,342.080145,249343.643634,334.718229,219979.578944,317.602263
